In [42]:
# from tensorflow_docs.vis import embed
from tensorflow import keras
# from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
# import imageio
from sklearn.model_selection import train_test_split
import cv2
import os

In [43]:
df = pd.read_csv("train.csv")
print(len(df))

# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")

train_df,test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

test_df.sample(10)

288
Total videos for training: 230
Total videos for testing: 58


,video_name,tag
256,0054.mp4,Real
183,27.mp4,Real
239,0038.mp4,Real
117,000057.mp4,Fake
46,000014.mp4,Fake
24,000012.mp4,Fake
250,0049.mp4,Real
237,0036.mp4,Real
113,000053.mp4,Fake
116,000056.mp4,Fake


In [65]:
IMG_SIZE = 150
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 30
NUM_FEATURES = 2048

In [45]:
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [46]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [47]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['Fake', 'Real']


In [48]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )


train_data, train_labels = prepare_all_videos(train_df, "../DeepFakeDetection/Dataset/train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (230, 30, 2048)
Frame masks in train set: (230, 30)


In [52]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "../Temp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/10000
6/6 [==============================] - 6s 311ms/step - loss: 0.6913 - accuracy: 0.5155 - val_loss: 0.6846 - val_accuracy: 0.5652

Epoch 00001: val_loss improved from inf to 0.68464, saving model to ../Temp\video_classifier
Epoch 2/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.6633 - accuracy: 0.6273 - val_loss: 0.6457 - val_accuracy: 0.7246

Epoch 00002: val_loss improved from 0.68464 to 0.64568, saving model to ../Temp\video_classifier
Epoch 3/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.6561 - accuracy: 0.6273 - val_loss: 0.6240 - val_accuracy: 0.6812

Epoch 00003: val_loss improved from 0.64568 to 0.62395, saving model to ../Temp\video_classifier
Epoch 4/10000
6/6 [==============================] - 0s 22ms/step - loss: 0.6423 - accuracy: 0.6522 - val_loss: 0.5772 - val_accuracy: 0.7246

Epoch 00004: val_loss improved from 0.62395 to 0.57721, saving model to ../Temp\video_classifier
Epoch 5/10000
6/6 [========================

Epoch 39/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.2003 - accuracy: 0.9752 - val_loss: 0.4226 - val_accuracy: 0.7826

Epoch 00039: val_loss did not improve from 0.41517
Epoch 40/10000
6/6 [==============================] - 0s 24ms/step - loss: 0.2175 - accuracy: 0.9627 - val_loss: 0.4344 - val_accuracy: 0.7971

Epoch 00040: val_loss did not improve from 0.41517
Epoch 41/10000
6/6 [==============================] - 0s 28ms/step - loss: 0.1935 - accuracy: 0.9814 - val_loss: 0.4287 - val_accuracy: 0.7971

Epoch 00041: val_loss did not improve from 0.41517
Epoch 42/10000
6/6 [==============================] - 0s 29ms/step - loss: 0.1895 - accuracy: 0.9503 - val_loss: 0.4289 - val_accuracy: 0.7971

Epoch 00042: val_loss did not improve from 0.41517
Epoch 43/10000
6/6 [==============================] - 0s 28ms/step - loss: 0.1453 - accuracy: 0.9876 - val_loss: 0.4349 - val_accuracy: 0.7971

Epoch 00043: val_loss did not improve from 0.41517
Epoch 44/10000
6/6 [=====

6/6 [==============================] - 0s 24ms/step - loss: 0.0296 - accuracy: 1.0000 - val_loss: 0.8441 - val_accuracy: 0.7826

Epoch 00081: val_loss did not improve from 0.41517
Epoch 82/10000
6/6 [==============================] - 0s 25ms/step - loss: 0.0453 - accuracy: 0.9938 - val_loss: 0.8365 - val_accuracy: 0.7971

Epoch 00082: val_loss did not improve from 0.41517
Epoch 83/10000
6/6 [==============================] - 0s 24ms/step - loss: 0.0322 - accuracy: 0.9938 - val_loss: 0.8374 - val_accuracy: 0.7971

Epoch 00083: val_loss did not improve from 0.41517
Epoch 84/10000
6/6 [==============================] - 0s 26ms/step - loss: 0.0343 - accuracy: 0.9938 - val_loss: 0.8403 - val_accuracy: 0.7971

Epoch 00084: val_loss did not improve from 0.41517
Epoch 85/10000
6/6 [==============================] - 0s 25ms/step - loss: 0.0309 - accuracy: 1.0000 - val_loss: 0.8434 - val_accuracy: 0.7971

Epoch 00085: val_loss did not improve from 0.41517
Epoch 86/10000
6/6 [====================

6/6 [==============================] - 0s 24ms/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 1.0817 - val_accuracy: 0.7826

Epoch 00123: val_loss did not improve from 0.41517
Epoch 124/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0367 - accuracy: 0.9876 - val_loss: 1.0876 - val_accuracy: 0.7826

Epoch 00124: val_loss did not improve from 0.41517
Epoch 125/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0169 - accuracy: 1.0000 - val_loss: 1.0918 - val_accuracy: 0.7826

Epoch 00125: val_loss did not improve from 0.41517
Epoch 126/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0239 - accuracy: 0.9938 - val_loss: 1.0953 - val_accuracy: 0.7826

Epoch 00126: val_loss did not improve from 0.41517
Epoch 127/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0185 - accuracy: 0.9938 - val_loss: 1.1001 - val_accuracy: 0.7971

Epoch 00127: val_loss did not improve from 0.41517
Epoch 128/10000
6/6 [===============

6/6 [==============================] - 0s 23ms/step - loss: 0.0111 - accuracy: 0.9938 - val_loss: 1.2996 - val_accuracy: 0.7826

Epoch 00165: val_loss did not improve from 0.41517
Epoch 166/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 1.3042 - val_accuracy: 0.7971

Epoch 00166: val_loss did not improve from 0.41517
Epoch 167/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 1.3087 - val_accuracy: 0.7971

Epoch 00167: val_loss did not improve from 0.41517
Epoch 168/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 1.3153 - val_accuracy: 0.7826

Epoch 00168: val_loss did not improve from 0.41517
Epoch 169/10000
6/6 [==============================] - 0s 23ms/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 1.3185 - val_accuracy: 0.7826

Epoch 00169: val_loss did not improve from 0.41517
Epoch 170/10000
6/6 [===============

...

6/6 [==============================] - 0s 24ms/step - loss: 1.6883e-05 - accuracy: 1.0000 - val_loss: 5.4177 - val_accuracy: 0.7681

Epoch 09820: val_loss did not improve from 0.41517
Epoch 9821/10000
6/6 [==============================] - 0s 25ms/step - loss: 6.9706e-06 - accuracy: 1.0000 - val_loss: 5.4221 - val_accuracy: 0.7681

Epoch 09821: val_loss did not improve from 0.41517
Epoch 9822/10000
6/6 [==============================] - 0s 26ms/step - loss: 4.9298e-06 - accuracy: 1.0000 - val_loss: 5.4244 - val_accuracy: 0.7681

Epoch 09822: val_loss did not improve from 0.41517
Epoch 9823/10000
6/6 [==============================] - 0s 30ms/step - loss: 5.9344e-06 - accuracy: 1.0000 - val_loss: 5.4260 - val_accuracy: 0.7681

Epoch 09823: val_loss did not improve from 0.41517
Epoch 9824/10000
6/6 [==============================] - 0s 30ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 5.4200 - val_accuracy: 0.7681

Epoch 09824: val_loss did not improve from 0.41517
Epoch 9825/10000

6/6 [==============================] - 0s 24ms/step - loss: 2.7470e-07 - accuracy: 1.0000 - val_loss: 5.6022 - val_accuracy: 0.7536

Epoch 09861: val_loss did not improve from 0.41517
Epoch 9862/10000
6/6 [==============================] - 0s 25ms/step - loss: 5.1732e-06 - accuracy: 1.0000 - val_loss: 5.6026 - val_accuracy: 0.7536

Epoch 09862: val_loss did not improve from 0.41517
Epoch 9863/10000
6/6 [==============================] - 0s 24ms/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 5.5823 - val_accuracy: 0.7536

Epoch 09863: val_loss did not improve from 0.41517
Epoch 9864/10000
6/6 [==============================] - 0s 25ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 5.5606 - val_accuracy: 0.7536

Epoch 09864: val_loss did not improve from 0.41517
Epoch 9865/10000
6/6 [==============================] - 0s 24ms/step - loss: 1.6880e-06 - accuracy: 1.0000 - val_loss: 5.5442 - val_accuracy: 0.7536

Epoch 09865: val_loss did not improve from 0.41517
Epoch 9866/10000
6/6

6/6 [==============================] - 0s 25ms/step - loss: 1.7546e-06 - accuracy: 1.0000 - val_loss: 5.5678 - val_accuracy: 0.7536

Epoch 09902: val_loss did not improve from 0.41517
Epoch 9903/10000
6/6 [==============================] - 0s 25ms/step - loss: 2.3165e-05 - accuracy: 1.0000 - val_loss: 5.5696 - val_accuracy: 0.7536

Epoch 09903: val_loss did not improve from 0.41517
Epoch 9904/10000
6/6 [==============================] - 0s 25ms/step - loss: 3.6789e-05 - accuracy: 1.0000 - val_loss: 5.5737 - val_accuracy: 0.7536

Epoch 09904: val_loss did not improve from 0.41517
Epoch 9905/10000
6/6 [==============================] - 0s 25ms/step - loss: 2.3014e-05 - accuracy: 1.0000 - val_loss: 5.5698 - val_accuracy: 0.7536

Epoch 09905: val_loss did not improve from 0.41517
Epoch 9906/10000
6/6 [==============================] - 0s 24ms/step - loss: 1.8363e-07 - accuracy: 1.0000 - val_loss: 5.5423 - val_accuracy: 0.7536

Epoch 09906: val_loss did not improve from 0.41517
Epoch 9907/1

6/6 [==============================] - 0s 24ms/step - loss: 1.3254e-07 - accuracy: 1.0000 - val_loss: 5.5206 - val_accuracy: 0.7536

Epoch 09943: val_loss did not improve from 0.41517
Epoch 9944/10000
6/6 [==============================] - 0s 24ms/step - loss: 1.2309e-05 - accuracy: 1.0000 - val_loss: 5.5255 - val_accuracy: 0.7536

Epoch 09944: val_loss did not improve from 0.41517
Epoch 9945/10000
6/6 [==============================] - 0s 25ms/step - loss: 4.4552e-06 - accuracy: 1.0000 - val_loss: 5.5296 - val_accuracy: 0.7536

Epoch 09945: val_loss did not improve from 0.41517
Epoch 9946/10000
6/6 [==============================] - 0s 24ms/step - loss: 7.4614e-06 - accuracy: 1.0000 - val_loss: 5.5319 - val_accuracy: 0.7536

Epoch 09946: val_loss did not improve from 0.41517
Epoch 9947/10000
6/6 [==============================] - 0s 24ms/step - loss: 7.8485e-08 - accuracy: 1.0000 - val_loss: 5.5331 - val_accuracy: 0.7536

Epoch 09947: val_loss did not improve from 0.41517
Epoch 9948/1

6/6 [==============================] - 0s 22ms/step - loss: 9.3802e-06 - accuracy: 1.0000 - val_loss: 5.6479 - val_accuracy: 0.7536

Epoch 09984: val_loss did not improve from 0.41517
Epoch 9985/10000
6/6 [==============================] - 0s 22ms/step - loss: 1.5457e-05 - accuracy: 1.0000 - val_loss: 5.6328 - val_accuracy: 0.7536

Epoch 09985: val_loss did not improve from 0.41517
Epoch 9986/10000
6/6 [==============================] - 0s 23ms/step - loss: 3.4800e-08 - accuracy: 1.0000 - val_loss: 5.6258 - val_accuracy: 0.7536

Epoch 09986: val_loss did not improve from 0.41517
Epoch 9987/10000
6/6 [==============================] - 0s 22ms/step - loss: 1.3267e-06 - accuracy: 1.0000 - val_loss: 5.6221 - val_accuracy: 0.7536

Epoch 09987: val_loss did not improve from 0.41517
Epoch 9988/10000
6/6 [==============================] - 0s 23ms/step - loss: 6.2079e-06 - accuracy: 1.0000 - val_loss: 5.6211 - val_accuracy: 0.7536

Epoch 09988: val_loss did not improve from 0.41517
Epoch 9989/1

In [53]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

In [64]:
test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)

Test video path: 000051.mp4
  Real: 52.93%
  Fake: 47.07%
